# XGBoost Full Pipeline: Training with Enhanced Features

This notebook walks through the complete XGBoost training pipeline with all the enhanced features:
- Review sentiment analysis
- User behavioral profiling
- Cuisine complementarity
- Operating hours analysis
- Topic extraction
- Service options

**Inputs (from SSD):**
- `/Volumes/SunnySSD/review-Georgia.json` (7.2GB)
- `/Volumes/SunnySSD/meta-Georgia.json` (168MB)

**Outputs (to SSD by default):**
- Processed Parquet files in `/Volumes/SunnySSD/Forkast_processed/ga/` (or local if SSD unavailable)
- Cached review analysis and user profiles for faster subsequent runs
- Trained XGBoost model
- Feature importance analysis
- Transition probability matrix
- Evaluation metrics

**Note:** Set `USE_SSD_FOR_OUTPUTS = False` in the setup cell to use local disk instead of SSD.


## Setup: Paths and Imports


In [ ]:
import sys
from pathlib import Path
import polars as pl
import pandas as pd
import numpy as np
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Add src to path
base_dir = Path.cwd().parent
sys.path.append(str(base_dir / 'src'))

# ============================================================================
# CONFIGURATION: Choose where to save processed data
# ============================================================================
USE_SSD_FOR_OUTPUTS = True  # Set to False to use local disk instead

# Define paths
SSD_PATH = Path("/Volumes/SunnySSD")
BASE_PROCESSED_DIR = base_dir / "data" / "processed" / "ga"

# Choose output directory based on configuration
if USE_SSD_FOR_OUTPUTS and SSD_PATH.exists():
    PROCESSED_DIR = SSD_PATH / "Forkast_processed" / "ga"
    print(f" Using SSD for processed data output: {PROCESSED_DIR}")
else:
    PROCESSED_DIR = BASE_PROCESSED_DIR
    if USE_SSD_FOR_OUTPUTS:
        print(f"  SSD not found at {SSD_PATH}, falling back to local: {PROCESSED_DIR}")
    else:
        print(f"Using local disk for processed data output: {PROCESSED_DIR}")

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# SSD input files (raw data)
REVIEW_JSON = SSD_PATH / "review-Georgia.json"
META_JSON = SSD_PATH / "meta-Georgia.json"

# Output paths (will use SSD if USE_SSD_FOR_OUTPUTS=True)
REVIEWS_PARQUET = PROCESSED_DIR / "reviews_ga.parquet"
BIZ_PARQUET = PROCESSED_DIR / "biz_ga.parquet"

# Cache directory for review analysis (can also go to SSD)
CACHE_DIR = PROCESSED_DIR / "cache"

print("\n✓ Setup complete")
print(f"  Base directory: {base_dir}")
print(f"  SSD input path: {SSD_PATH}")
print(f"  Processed output: {PROCESSED_DIR}")
print(f"  Cache directory: {CACHE_DIR}")
print(f"  Review JSON exists: {REVIEW_JSON.exists()}")
print(f"  Meta JSON exists: {META_JSON.exists()}")
print(f"\n💡 Tip: Set USE_SSD_FOR_OUTPUTS=False to use local disk instead")


## Phase A1: Data Ingestion & Normalization

**What this does:**
- Loads raw JSON files from SSD
- Filters to Georgia geographic bounds
- Normalizes categories and prices
- **Extracts operating hours and service options** (NEW!)
- Converts to efficient Parquet format

**Expected time:** 10-20 minutes for 7GB review file


In [2]:
# Reload the module to get the latest changes
import importlib
import data.ingest
importlib.reload(data.ingest)
from data.ingest import ingest_metadata, ingest_reviews

print("=" * 80)
print("PHASE A1: DATA INGESTION")
print("=" * 80)

# Ingest metadata
print("\n[1/2] Ingesting metadata...")
biz_df, valid_ids = ingest_metadata(
    str(META_JSON),
    str(BIZ_PARQUET)
)

print(f"\n✓ Metadata ingested:")
print(f"  Total businesses: {len(biz_df):,}")
print(f"  Columns: {list(biz_df.columns)}")
print(f"  New metadata columns: operating_hours_parsed, has_delivery, has_takeout, etc.")


PHASE A1: DATA INGESTION

[1/2] Ingesting metadata...

PHASE A1: INGESTING METADATA
Input: /Volumes/SunnySSD/meta-Georgia.json
Output: /Volumes/SunnySSD/Forkast_processed/ga/biz_ga.parquet

[1/6] Reading JSON file...
  Loaded 166,381 raw businesses

[2/6] Filtering to Georgia geographic bounds...
  Retained 166,334 businesses in Georgia

[3/6] Parsing price buckets...

[4/6] Detecting closed businesses...

[5/9] Normalizing categories...

[6/9] Filtering to food-only businesses...
  Retained 27,757 food-related businesses

[7/9] Parsing operating hours...

[8/9] Extracting service options...

[9/9] Finalizing schema...
  Final count: 27,710 unique businesses

Writing to /Volumes/SunnySSD/Forkast_processed/ga/biz_ga.parquet...

✓ Metadata ingestion complete!
  Output size: 5.4 MB

✓ Metadata ingested:
  Total businesses: 27,710
  Columns: ['gmap_id', 'name', 'lat', 'lon', 'category_main', 'category_all', 'avg_rating', 'num_reviews', 'price_bucket', 'is_closed', 'relative_results', 'oper

In [3]:
# Preview metadata with new columns
print("\nSample metadata with new columns:")
print(biz_df.select([
    "gmap_id", "name", "category_main", "has_delivery", 
    "has_takeout", "has_late_night", "is_24hr", "days_open_count"
]).head(5))



Sample metadata with new columns:
shape: (5, 8)
┌────────────┬────────────┬────────────┬────────────┬────────────┬───────────┬─────────┬───────────┐
│ gmap_id    ┆ name       ┆ category_m ┆ has_delive ┆ has_takeou ┆ has_late_ ┆ is_24hr ┆ days_open │
│ ---        ┆ ---        ┆ ain        ┆ ry         ┆ t          ┆ night     ┆ ---     ┆ _count    │
│ str        ┆ str        ┆ ---        ┆ ---        ┆ ---        ┆ ---       ┆ bool    ┆ ---       │
│            ┆            ┆ str        ┆ bool       ┆ bool       ┆ bool      ┆         ┆ i8        │
╞════════════╪════════════╪════════════╪════════════╪════════════╪═══════════╪═════════╪═══════════╡
│ 0x88fb9f97 ┆ Coco and   ┆ restaurant ┆ true       ┆ true       ┆ true      ┆ false   ┆ 5         │
│ aca0a781:0 ┆ Moss       ┆            ┆            ┆            ┆           ┆         ┆           │
│ xc7c75f293 ┆            ┆            ┆            ┆            ┆           ┆         ┆           │
│ …          ┆            ┆            ┆  

In [4]:
# Ingest reviews
print("\n[2/2] Ingesting reviews...")
print("  This may take 10-20 minutes for the 7GB file...")

reviews_df = ingest_reviews(
    str(REVIEW_JSON),
    str(REVIEWS_PARQUET),
    biz_gmap_ids=valid_ids
)

print(f"\n✓ Reviews ingested:")
print(f"  Total reviews: {len(reviews_df):,}")
print(f"  Unique users: {reviews_df['user_id'].n_unique():,}")
print(f"  Unique restaurants: {reviews_df['gmap_id'].n_unique():,}")
print(f"  Date range: {reviews_df['ts'].min()} to {reviews_df['ts'].max()}")



[2/2] Ingesting reviews...
  This may take 10-20 minutes for the 7GB file...
PHASE A1: INGESTING REVIEWS
Input: /Volumes/SunnySSD/review-Georgia.json
Output: /Volumes/SunnySSD/Forkast_processed/ga/reviews_ga.parquet

[1/5] Reading JSON file...


  Loaded 24,060,125 raw reviews

[2/5] Converting timestamps...

[3/5] Filtering invalid timestamps...
  Retained 24,060,120 reviews with valid timestamps

[4/5] Creating derived columns...

  Filtering out reviews with missing user_id or rating...
  Removed 167,538 reviews with null user_id or rating
  Retained 23,892,582 reviews

  Filtering to 27,710 valid businesses...
  Retained 10,494,609 reviews

[5/5] Deduplicating...
  Final count: 10,339,035 unique reviews

Writing to /Volumes/SunnySSD/Forkast_processed/ga/reviews_ga.parquet...

✓ Reviews ingestion complete!
  Output size: 787.2 MB

✓ Reviews ingested:
  Total reviews: 10,339,035
  Unique users: 2,546,362
  Unique restaurants: 27,710
  Date range: 2001-01-06 00:00:00 to 2021-09-08 01:43:37


## Phase A2: User Sequence Derivation

**What this does:**
- Creates user visit sequences sorted by timestamp
- Generates consecutive visit pairs (A → B)
- Filters pairs within 0-168 hours (1 week window)

**Expected time:** 5-10 minutes


In [5]:
# Reload module to get latest changes
import importlib
import data.sequences
importlib.reload(data.sequences)
from data.sequences import main as sequences_main

print("=" * 80)
print("PHASE A2: USER SEQUENCE DERIVATION")
print("=" * 80)
print(f"\nUsing processed directory: {PROCESSED_DIR}")

# Run sequence derivation with explicit path
# This ensures it uses the same PROCESSED_DIR as configured in setup
sequences_main(processed_dir=PROCESSED_DIR)

# Load and preview results
pairs_df = pl.read_parquet(PROCESSED_DIR / "pairs_ga.parquet")

print(f"\n✓ Sequences created:")
print(f"  Total pairs: {len(pairs_df):,}")
print(f"  Unique source restaurants: {pairs_df['src_gmap_id'].n_unique():,}")
print(f"  Unique destination restaurants: {pairs_df['dst_gmap_id'].n_unique():,}")
print(f"  Average time gap: {pairs_df['delta_hours'].mean():.1f} hours")


PHASE A2: USER SEQUENCE DERIVATION

Using processed directory: /Volumes/SunnySSD/Forkast_processed/ga
Loading data...
  Loaded 10,339,035 reviews
  Loaded 27,710 businesses
DERIVING USER SEQUENCES

[1/4] Joining reviews with business metadata...
  Joined 10,339,035 reviews with business data

[2/4] Sorting by user and timestamp...

[3/4] Creating sequence indices...

[4/4] Final sequence count: 10,339,035

Writing to /Volumes/SunnySSD/Forkast_processed/ga/user_sequences_ga.parquet...
  Output size: 256.8 MB

DERIVING CONSECUTIVE PAIRS

[1/5] Creating shifted columns for next visit...

[2/5] Filtering out null destinations (last visit in sequence)...
  Retained 7,792,673 pairs

[3/5] Calculating time delta...

[4/5] Filtering by time window...
  Retained 4,152,155 pairs within 0-168 hour window

[5/5] Final pair count: 4,152,155

Writing to /Volumes/SunnySSD/Forkast_processed/ga/pairs_ga.parquet...
  Output size: 193.6 MB

SEQUENCE STATISTICS

USER SEQUENCES:
  Total visits: 10,339,035


In [6]:
# Preview sample pairs
print("\nSample consecutive visit pairs:")
print(pairs_df.select([
    "user_id", "src_gmap_id", "dst_gmap_id", 
    "delta_hours", "src_ts", "dst_ts"
]).head(10))



Sample consecutive visit pairs:
shape: (10, 6)
┌────────────────┬────────────────┬────────────────┬─────────────┬────────────────┬────────────────┐
│ user_id        ┆ src_gmap_id    ┆ dst_gmap_id    ┆ delta_hours ┆ src_ts         ┆ dst_ts         │
│ ---            ┆ ---            ┆ ---            ┆ ---         ┆ ---            ┆ ---            │
│ str            ┆ str            ┆ str            ┆ f64         ┆ datetime[μs]   ┆ datetime[μs]   │
╞════════════════╪════════════════╪════════════════╪═════════════╪════════════════╪════════════════╡
│ 10000000713488 ┆ 0x88f5758473a8 ┆ 0x88f5759cc6ea ┆ 0.019444    ┆ 2019-05-14     ┆ 2019-05-14     │
│ 6560887        ┆ bc37:0x9367635 ┆ f6a9:0x16a87fc ┆             ┆ 02:00:43       ┆ 02:01:53       │
│                ┆ ab…            ┆ b6…            ┆             ┆                ┆                │
│ 10000000713488 ┆ 0x88f5759cc6ea ┆ 0x88f59b65e7b2 ┆ 0.013333    ┆ 2019-05-14     ┆ 2019-05-14     │
│ 6560887        ┆ f6a9:0x16a87fc ┆ e153:0x

## Phase A2.5: Data Quality Filtering

**What this does:**
- Re-categorizes generic 'restaurant' businesses
- Filters to users with 5+ visits
- Removes pairs with very short time deltas

**Expected time:** 2-5 minutes


In [7]:
# Reload module to get latest changes
import importlib
import data.filter_quality
importlib.reload(data.filter_quality)
from data.filter_quality import main as filter_main

print("=" * 80)
print("PHASE A2.5: DATA QUALITY FILTERING")
print("=" * 80)
print(f"\nUsing processed directory: {PROCESSED_DIR}")

# Run quality filtering with explicit path
# This ensures it uses the same PROCESSED_DIR as configured in setup
filter_main(processed_dir=PROCESSED_DIR)

# Load filtered pairs
filtered_pairs = pl.read_parquet(PROCESSED_DIR / "pairs_filtered_ga.parquet")

print(f"\n✓ Quality filtering complete:")
print(f"  Filtered pairs: {len(filtered_pairs):,}")
print(f"  Retention rate: {len(filtered_pairs)/len(pairs_df)*100:.1f}%")


PHASE A2.5: DATA QUALITY FILTERING

Using processed directory: /Volumes/SunnySSD/Forkast_processed/ga
PHASE A2.5: DATA QUALITY FILTERING

Inputs:
  - /Volumes/SunnySSD/Forkast_processed/ga/biz_ga.parquet
  - /Volumes/SunnySSD/Forkast_processed/ga/user_sequences_ga.parquet
  - /Volumes/SunnySSD/Forkast_processed/ga/pairs_ga.parquet

Outputs:
  - /Volumes/SunnySSD/Forkast_processed/ga/biz_ga.parquet (updated)
  - /Volumes/SunnySSD/Forkast_processed/ga/user_sequences_filtered_ga.parquet
  - /Volumes/SunnySSD/Forkast_processed/ga/pairs_filtered_ga.parquet

[Loading data...]

STEP 1: RE-CATEGORIZING 'RESTAURANT' BUSINESSES

Original 'restaurant' businesses: 4,490

[1/3] Inferring categories from business names...

[2/3] Re-categorization results:
  Successfully re-categorized: 1,883
  Moved to 'other': 2,607

[3/3] New category distribution (top 15):
   1. fast_food           :  3,396 ( 12.3%)
   2. american            :  2,759 ( 10.0%)
   3. other               :  2,607 (  9.4%)
   4. mexi

## Phase A3: Feature Engineering (Enhanced!)

**What this does:**

**Original features (1-7):**
1. Spatial: distance, neighborhood, direction
2. Temporal: time gaps, day of week, meal type
3. Quality: ratings, rating differences
4. Price: price levels, price differences
5. Category: cuisine type matching
6. Relationship: relative_results ranking

**NEW Enhanced features (8-14):**
7. **Review Sentiment**: Average sentiment scores, sentiment transitions, review length
8. **User Behavioral**: User preferences, explorer flags, loyalty scores
9. **Cuisine Complementarity**: Complementary pairs (pizza→dessert), meal progressions
10. **Operating Hours**: Open/closed checks, hours overlap, late night transitions
11. **Topic Features**: Dessert/drinks mentions, topic transitions
12. **Service Options**: Delivery/takeout flags, service matching

**Expected time:** 15-30 minutes (includes sentiment analysis)


In [8]:
# Reload features module to get latest changes
import importlib
import data.features
importlib.reload(data.features)
from data.features import add_all_features, generate_negative_samples

print("=" * 80)
print("PHASE A3: ENHANCED FEATURE ENGINEERING")
print("=" * 80)

# Load data
pairs_df = pl.read_parquet(PROCESSED_DIR / "pairs_filtered_ga.parquet")
biz_df = pl.read_parquet(BIZ_PARQUET)
reviews_df = pl.read_parquet(REVIEWS_PARQUET)

print(f"\nLoaded data:")
print(f"  Pairs: {len(pairs_df):,}")
print(f"  Businesses: {len(biz_df):,}")
print(f"  Reviews: {len(reviews_df):,}")

print(f"\n[Step 1/3] Adding all features to positive pairs...")
print("  This includes sentiment analysis and user profiling...")
print(f"  Using cache directory: {CACHE_DIR}")

# Use analysis context to share cached results between positive and negative samples
analysis_context = {}
user_profile_cache = CACHE_DIR / "user_profiles.parquet"

pairs_with_features = add_all_features(
    pairs_df, 
    biz_df, 
    reviews_df,
    review_cache_path=CACHE_DIR,
    user_profile_cache_path=user_profile_cache,
    max_reviews_per_biz=200,  # Limit reviews per business for faster processing
    analysis_context=analysis_context
)

print(f"\n✓ Features added to {len(pairs_with_features):,} positive pairs")
print(f"  Total features: {len(pairs_with_features.columns)}")


PHASE A3: ENHANCED FEATURE ENGINEERING

Loaded data:
  Pairs: 985,006
  Businesses: 27,710
  Reviews: 10,339,035

[Step 1/3] Adding all features to positive pairs...
  This includes sentiment analysis and user profiling...
  Using cache directory: /Volumes/SunnySSD/Forkast_processed/ga/cache

ADDING FEATURES TO POSITIVE PAIRS

REVIEW ANALYSIS
  ✓ Loaded cached review analysis

✓ Review analysis complete!
  Sentiment features: 27,710 restaurants
  Topic features: 27,710 restaurants

USER PROFILING (cached)
  ✓ Loaded cached user profiles (1,365,435 users)

[1/7] Adding spatial features...
  - Calculating haversine distances...
  - Creating distance buckets...
  - Calculating directions...
  ✓ Added spatial features

[2/7] Adding temporal features...
  - Creating time delta buckets...
  - Extracting hour and day of week...
  - Identifying meal times...
  ✓ Added temporal features

[3/7] Adding quality features...
  ✓ Added quality features

[4/7] Adding price features...
  - Joining with

In [9]:
# Preview new features
print("\nNew Enhanced Features Preview:")
new_feature_cols = [
    "src_sentiment_score", "dst_sentiment_score", "sentiment_transition",
    "user_is_explorer", "user_visit_frequency", "cuisine_pair_type",
    "is_dessert_followup", "hours_overlap", "is_late_night_transition",
    "topic_transition_dessert", "service_match"
]

# Get columns that exist
existing_cols = [col for col in new_feature_cols if col in pairs_with_features.columns]

if existing_cols:
    print(pairs_with_features.select(existing_cols).head(5))
else:
    print("  Checking available columns...")
    all_cols = pairs_with_features.columns
    print(f"  Total columns: {len(all_cols)}")
    print(f"  Sample columns: {all_cols[:20]}")



New Enhanced Features Preview:
shape: (5, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ src_senti ┆ dst_senti ┆ sentiment ┆ user_is_e ┆ … ┆ hours_ove ┆ is_late_n ┆ topic_tra ┆ service_ │
│ ment_scor ┆ ment_scor ┆ _transiti ┆ xplorer   ┆   ┆ rlap      ┆ ight_tran ┆ nsition_d ┆ match    │
│ e         ┆ e         ┆ on        ┆ ---       ┆   ┆ ---       ┆ sition    ┆ essert    ┆ ---      │
│ ---       ┆ ---       ┆ ---       ┆ bool      ┆   ┆ bool      ┆ ---       ┆ ---       ┆ bool     │
│ f32       ┆ f32       ┆ f32       ┆           ┆   ┆           ┆ bool      ┆ bool      ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0.394242  ┆ 0.226775  ┆ -0.167467 ┆ false     ┆ … ┆ true      ┆ false     ┆ false     ┆ true     │
│ 0.415869  ┆ 0.363042  ┆ -0.052827 ┆ false     ┆ … ┆ true      ┆ false     ┆ false     ┆ false    │
│ 0.375567  ┆ 0.389334  ┆ 0.013767  ┆ false 

In [10]:
# Generate negative samples and add features (full pipeline)
print("\n[Step 2/3] Generating negative samples with features...")
print("  Generating 4 negative samples per positive pair...")
print(f"  Using cached analysis data from previous step...")

# Reuse cached analysis context from positive pairs generation
# (analysis_context should be available from cell 13)
user_profile_cache = CACHE_DIR / "user_profiles.parquet"

negative_pairs = generate_negative_samples(
    pairs_with_features, 
    biz_df, 
    n_negatives=4,
    random_seed=42,
    sentiment_df=analysis_context.get("sentiment_df"),
    topics_df=analysis_context.get("topics_df"),
    user_profiles=analysis_context.get("user_profiles_df"),
    reviews_df=analysis_context.get("reviews_subset_df"),
    review_cache_path=CACHE_DIR,
    user_profile_cache_path=user_profile_cache,
    max_reviews_per_biz=200
)

print(f"\n✓ Generated {len(negative_pairs):,} negative pairs")
print(f"  ✓ Features already added during generation (using cached analysis)")



[Step 2/3] Generating negative samples with features...
  Generating 4 negative samples per positive pair...
  Using cached analysis data from previous step...

GENERATING NEGATIVE SAMPLES (4:1 ratio)

Negative sample distribution per positive:
  - Geographic (10km): 2
  - Relative results: 1
  - Same category: 1
  - Total: 4

[1/4] Creating business lookup structures...
  ✓ Indexed 27,710 businesses

[2/4] Generating 3,940,024 negative samples...
  (This will take ~985 minutes for 985,006 positive pairs)
  Progress: 5,000/985,006 (0.5%) - ETA: 5.7 min
  Progress: 10,000/985,006 (1.0%) - ETA: 5.8 min
  Progress: 15,000/985,006 (1.5%) - ETA: 5.8 min
  Progress: 20,000/985,006 (2.0%) - ETA: 5.8 min
  Progress: 25,000/985,006 (2.5%) - ETA: 5.8 min
  Progress: 30,000/985,006 (3.0%) - ETA: 5.7 min
  Progress: 35,000/985,006 (3.6%) - ETA: 5.7 min
  Progress: 40,000/985,006 (4.1%) - ETA: 5.7 min
  Progress: 45,000/985,006 (4.6%) - ETA: 5.6 min
  Progress: 50,000/985,006 (5.1%) - ETA: 5.6 min

In [11]:
# Combine and save features
print("\n[Step 3/3] Combining positive and negative samples...")

# Ensure schema compatibility
target_schema = pairs_with_features.schema
target_columns = set(target_schema.keys())

# First, select only columns that exist in target schema (drop extras)
columns_to_select = [col for col in negative_pairs.columns if col in target_columns]
negative_pairs = negative_pairs.select(columns_to_select)

# Now ensure all target columns exist and have correct types
cast_exprs = []
for col_name, dtype in target_schema.items():
    if col_name in negative_pairs.columns:
        if negative_pairs[col_name].dtype != dtype:
            cast_exprs.append(pl.col(col_name).cast(dtype))
        else:
            cast_exprs.append(pl.col(col_name))
    else:
        # Fill missing columns with defaults
        if dtype == pl.Boolean:
            cast_exprs.append(pl.lit(False).cast(dtype).alias(col_name))
        elif dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64]:
            cast_exprs.append(pl.lit(0).cast(dtype).alias(col_name))
        elif dtype in [pl.Float32, pl.Float64]:
            cast_exprs.append(pl.lit(0.0).cast(dtype).alias(col_name))
        else:
            cast_exprs.append(pl.lit(None).cast(dtype).alias(col_name))

if cast_exprs:
    negative_pairs = negative_pairs.with_columns(cast_exprs)

# Ensure column order matches target schema
negative_pairs = negative_pairs.select(list(target_schema.keys()))

# Combine
all_features_df = pl.concat([pairs_with_features, negative_pairs])

features_output = PROCESSED_DIR / "features_ga.parquet"
all_features_df.write_parquet(features_output, compression="snappy")

print(f"\n✓ Features saved:")
print(f"  Total pairs (pos + neg): {len(all_features_df):,}")
print(f"  Positive: {len(pairs_with_features):,}")
print(f"  Negative: {len(negative_pairs):,}")
print(f"  File: {features_output}")
print(f"  Size: {features_output.stat().st_size / 1024 / 1024:.1f} MB")



[Step 3/3] Combining positive and negative samples...

✓ Features saved:
  Total pairs (pos + neg): 4,363,086
  Positive: 985,006
  Negative: 3,378,080
  File: /Volumes/SunnySSD/Forkast_processed/ga/features_ga.parquet
  Size: 640.9 MB


## Phase A4: Temporal Data Splitting

**What this does:**
- Splits data chronologically by `src_ts` timestamp
- 70% train, 15% validation, 15% test
- Ensures no data leakage (future data not in training)

**Expected time:** 2-5 minutes


In [12]:
# Reload module to get latest changes
import importlib
import data.split_data
importlib.reload(data.split_data)
from data.split_data import split_xgboost_only

print("=" * 80)
print("PHASE A4: TEMPORAL DATA SPLITTING (XGBOOST ONLY)")
print("=" * 80)
print(f"\nUsing processed directory: {PROCESSED_DIR}")

# Run data splitting (XGBoost only - no LSTM data) with explicit path
# This ensures it uses the same PROCESSED_DIR as configured in setup
split_xgboost_only(processed_dir=PROCESSED_DIR)

# Load split data
train_df = pl.read_parquet(PROCESSED_DIR / "xgboost_data" / "train.parquet")
val_df = pl.read_parquet(PROCESSED_DIR / "xgboost_data" / "val.parquet")
test_df = pl.read_parquet(PROCESSED_DIR / "xgboost_data" / "test.parquet")

print(f"\n✓ Data split complete:")
print(f"  Train: {len(train_df):,} pairs ({len(train_df)/(len(train_df)+len(val_df)+len(test_df))*100:.1f}%)")
print(f"  Validation: {len(val_df):,} pairs ({len(val_df)/(len(train_df)+len(val_df)+len(test_df))*100:.1f}%)")
print(f"  Test: {len(test_df):,} pairs ({len(test_df)/(len(train_df)+len(val_df)+len(test_df))*100:.1f}%)")
print(f"\n  Positive/negative ratio:")
for name, df in [("Train", train_df), ("Val", val_df), ("Test", test_df)]:
    pos_count = df["label"].sum() if "label" in df.columns else 0
    neg_count = len(df) - pos_count
    print(f"    {name}: {pos_count:,} positive, {neg_count:,} negative ({pos_count/len(df)*100:.1f}% positive)")


PHASE A4: TEMPORAL DATA SPLITTING (XGBOOST ONLY)

Using processed directory: /Volumes/SunnySSD/Forkast_processed/ga

[Loading data...]
  XGBoost: 4,363,086 samples

[Splitting XGBoost data...]

SPLITTING XGBOOST DATA (TEMPORAL)

[1/4] Sorting by timestamp...

[2/4] Calculating split indices...
  Total samples: 4,363,086
  Train size: 3,054,160 (70.0%)
  Val size: 654,462 (15.0%)
  Test size: 654,464 (15.0%)

[3/4] Splitting data...

[4/4] Temporal ranges:
  Train: 2005-12-09 00:00:00 to 2019-11-11 10:04:14
  Val:   2019-11-11 10:04:14 to 2020-08-23 14:26:59
  Test:  2020-08-23 14:26:59 to 2021-09-04 12:21:12

  Label distribution:
  Train - Pos: 685,853, Neg: 2,368,307
  Val   - Pos: 148,053, Neg: 506,409
  Test  - Pos: 151,100, Neg: 503,364

[Saving XGBoost splits...]
  ✓ Saved train.parquet (3,054,160 samples)
  ✓ Saved val.parquet (654,462 samples)
  ✓ Saved test.parquet (654,464 samples)

[Copying business metadata...]
  ✓ Copied biz_ga.parquet to xgboost_data/

✓ Data split comple

In [ ]:
# STEP 1: Regenerate features with fixed negative sampling
print("=" * 80)
print("REGENERATING FEATURES WITH FIXED NEGATIVE SAMPLING")
print("=" * 80)

# Reload the updated modules
import importlib
import data.features
importlib.reload(data.features)
from data.features import add_all_features, generate_negative_samples

# Load the filtered pairs (before feature engineering)
pairs_df = pl.read_parquet(PROCESSED_DIR / "pairs_filtered_ga.parquet")
biz_df = pl.read_parquet(BIZ_PARQUET)
reviews_df = pl.read_parquet(REVIEWS_PARQUET)

print(f"\nLoaded data:")
print(f"  Pairs: {len(pairs_df):,}")
print(f"  Businesses: {len(biz_df):,}")
print(f"  Reviews: {len(reviews_df):,}")

# Add features to positive pairs with caching
print(f"\n[1/3] Adding features to positive pairs...")
print(f"  Using cache directory: {CACHE_DIR}")

analysis_context = {}
user_profile_cache = CACHE_DIR / "user_profiles.parquet"

pairs_with_features = add_all_features(
    pairs_df, 
    biz_df, 
    reviews_df,
    review_cache_path=CACHE_DIR,
    user_profile_cache_path=user_profile_cache,
    max_reviews_per_biz=200,
    analysis_context=analysis_context
)

# Generate negative samples with FIXED timestamps, reusing cached analysis
print(f"\n[2/3] Generating negative samples with realistic timestamps...")
negative_pairs = generate_negative_samples(
    pairs_with_features, 
    biz_df, 
    n_negatives=4, 
    random_seed=42,
    sentiment_df=analysis_context.get("sentiment_df"),
    topics_df=analysis_context.get("topics_df"),
    user_profiles=analysis_context.get("user_profiles_df"),
    reviews_df=analysis_context.get("reviews_subset_df"),
    review_cache_path=CACHE_DIR,
    user_profile_cache_path=user_profile_cache,
    max_reviews_per_biz=200
)

# Features are already added during negative sample generation when cached data is provided
print(f"\n[3/3] Combining positive and negative samples...")

# Combine and save
print(f"\nCombining positive and negative samples...")
# Ensure schema compatibility
target_schema = pairs_with_features.schema
target_columns = set(target_schema.keys())

# Select only columns that exist in target schema
columns_to_select = [col for col in negative_pairs.columns if col in target_columns]
negative_pairs = negative_pairs.select(columns_to_select)

# Ensure all target columns exist and have correct types
cast_exprs = []
for col_name, dtype in target_schema.items():
    if col_name in negative_pairs.columns:
        if negative_pairs[col_name].dtype != dtype:
            cast_exprs.append(pl.col(col_name).cast(dtype))
        else:
            cast_exprs.append(pl.col(col_name))
    else:
        # Fill missing columns with defaults
        if dtype == pl.Boolean:
            cast_exprs.append(pl.lit(False).cast(dtype).alias(col_name))
        elif dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64]:
            cast_exprs.append(pl.lit(0).cast(dtype).alias(col_name))
        elif dtype in [pl.Float32, pl.Float64]:
            cast_exprs.append(pl.lit(0.0).cast(dtype).alias(col_name))
        else:
            cast_exprs.append(pl.lit(None).cast(dtype).alias(col_name))

if cast_exprs:
    negative_pairs = negative_pairs.with_columns(cast_exprs)

# Ensure column order matches target schema
negative_pairs = negative_pairs.select(list(target_schema.keys()))

# Combine
all_features_df = pl.concat([pairs_with_features, negative_pairs])

# Save updated features
features_output = PROCESSED_DIR / "features_ga_fixed.parquet"
all_features_df.write_parquet(features_output, compression="snappy")

print(f"\n Fixed features saved:")
print(f"  Total pairs: {len(all_features_df):,}")
print(f"  Positive: {len(pairs_with_features):,}")
print(f"  Negative: {len(negative_pairs):,}")
print(f"  File: {features_output}")
print(f"  Size: {features_output.stat().st_size / 1024 / 1024:.1f} MB")


REGENERATING FEATURES WITH FIXED NEGATIVE SAMPLING

Loaded data:
  Pairs: 985,006
  Businesses: 27,710
  Reviews: 10,339,035

[1/3] Adding features to positive pairs...
  Using cache directory: /Volumes/SunnySSD/Forkast_processed/ga/cache

ADDING FEATURES TO POSITIVE PAIRS

REVIEW ANALYSIS
  ✓ Loaded cached review analysis

✓ Review analysis complete!
  Sentiment features: 27,710 restaurants
  Topic features: 27,710 restaurants

USER PROFILING (cached)
  ✓ Loaded cached user profiles (1,365,435 users)

[1/7] Adding spatial features...
  - Calculating haversine distances...
  - Creating distance buckets...
  - Calculating directions...
  ✓ Added spatial features

[2/7] Adding temporal features...
  - Creating time delta buckets...
  - Extracting hour and day of week...
  - Identifying meal times...
  ✓ Added temporal features

[3/7] Adding quality features...
  ✓ Added quality features

[4/7] Adding price features...
  - Joining with business data for prices...
  ✓ Added price features


In [ ]:
# STEP 2: Geographic splitting (Non-Atlanta train → Atlanta test)
print("=" * 80)
print("GEOGRAPHIC DATA SPLITTING")
print("=" * 80)

# Reload the updated split_data module
import data.split_data
importlib.reload(data.split_data)
from data.split_data import split_xgboost_geographic

# Use the fixed features file
import shutil
fixed_features = PROCESSED_DIR / "features_ga_fixed.parquet"
original_features = PROCESSED_DIR / "features_ga.parquet"

# Backup original and use fixed version
if original_features.exists():
    shutil.copy(original_features, PROCESSED_DIR / "features_ga_backup.parquet")
    print("  ✓ Backed up original features")

shutil.copy(fixed_features, original_features)
print("  ✓ Using fixed features for splitting")

# Run geographic splitting with explicit path
# This ensures it uses the same PROCESSED_DIR as configured in setup
print(f"\nUsing processed directory: {PROCESSED_DIR}")
train_df, val_df, test_df = split_xgboost_geographic(processed_dir=PROCESSED_DIR)

print(f"\n Geographic split complete:")
print(f"  Train (non-Atlanta): {len(train_df):,} pairs")
print(f"  Val (non-Atlanta): {len(val_df):,} pairs")
print(f"  Test (Atlanta): {len(test_df):,} pairs")


In [ ]:
# STEP 3: Retrain model with fixed data
print("=" * 80)
print("RETRAINING XGBOOST MODEL WITH FIXES")
print("=" * 80)

# Reload the updated XGBoost module
import models.xgboost_ranker
importlib.reload(models.xgboost_ranker)
from models.xgboost_ranker import (
    load_data, prepare_features, train_model, predict_ranking, 
    evaluate_ranking, analyze_feature_importance
)

# Load the geographically split data
data_dir = PROCESSED_DIR / "xgboost_data"
train_df, val_df, test_df = load_data(data_dir)

print(f"\n✓ Data loaded:")
print(f"  Train: {len(train_df):,} pairs")
print(f"  Validation: {len(val_df):,} pairs")
print(f"  Test: {len(test_df):,} pairs")

# Prepare features
print(f"\n[1/4] Preparing features...")
X_train, y_train, group_train, feature_names = prepare_features(train_df)
X_val, y_val, group_val, _ = prepare_features(val_df)
X_test, y_test, group_test, _ = prepare_features(test_df)

print(f"  Train shape: {X_train.shape}")
print(f"  Validation shape: {X_val.shape}")
print(f"  Test shape: {X_test.shape}")

# Train model
print(f"\n[2/4] Training model...")
model = train_model(X_train, y_train, group_train, X_val, y_val, group_val)

# Generate predictions
print(f"\n[3/4] Generating predictions...")
predictions_df = predict_ranking(model, test_df, top_k=10)

# Evaluate model (nDCG disabled by default to prevent kernel crashes)
print(f"\n[4/4] Evaluating model...")
metrics = evaluate_ranking(predictions_df, k_values=[1, 5, 10], compute_ndcg=False)

# Print final summary
print(f"\n" + "=" * 80)
print(f" FINAL MODEL PERFORMANCE")
print(f"=" * 80)
print(f"  Recall@1:  {metrics.get('recall@1', 0):.4f}")
print(f"  Recall@5:  {metrics.get('recall@5', 0):.4f}")
print(f"  Recall@10: {metrics.get('recall@10', 0):.4f}")
print(f"  MRR:       {metrics.get('mrr', 0):.4f}")

# Validate performance is realistic (not data leakage)
if metrics.get('recall@10', 0) < 0.9:
    print(f"\n Performance metrics are realistic - no data leakage detected")
else:
    print(f"\n  Very high performance detected - possible data leakage")


In [ ]:
# STEP 2: Geographic splitting (Non-Atlanta train → Atlanta test)
print("=" * 80)
print("GEOGRAPHIC DATA SPLITTING")
print("=" * 80)

# Reload the updated split_data module
import data.split_data
importlib.reload(data.split_data)
from data.split_data import split_xgboost_geographic

# Use the fixed features file
import shutil
fixed_features = PROCESSED_DIR / "features_ga_fixed.parquet"
original_features = PROCESSED_DIR / "features_ga.parquet"

# Backup original and use fixed version
if original_features.exists():
    shutil.copy(original_features, PROCESSED_DIR / "features_ga_backup.parquet")
    print("  ✓ Backed up original features")

shutil.copy(fixed_features, original_features)
print("  ✓ Using fixed features for splitting")

# Run geographic splitting with explicit path
# This ensures it uses the same PROCESSED_DIR as configured in setup
print(f"\nUsing processed directory: {PROCESSED_DIR}")
train_df, val_df, test_df = split_xgboost_geographic(processed_dir=PROCESSED_DIR)

print(f"\n Geographic split complete:")
print(f"  Train (non-Atlanta): {len(train_df):,} pairs")
print(f"  Val (non-Atlanta): {len(val_df):,} pairs")
print(f"  Test (Atlanta): {len(test_df):,} pairs")

# Verify the split worked correctly
atlanta_bounds = {
    'lat_min': 33.6, 'lat_max': 34.0,
    'lon_min': -84.6, 'lon_max': -84.2
}

# Check test set is Atlanta-only
test_biz_ids = set(test_df['src_gmap_id'].to_list() + test_df['dst_gmap_id'].to_list())
atlanta_test_biz = biz_df.filter(
    pl.col('gmap_id').is_in(list(test_biz_ids)) &
    (pl.col('lat') >= atlanta_bounds['lat_min']) &
    (pl.col('lat') <= atlanta_bounds['lat_max']) &
    (pl.col('lon') >= atlanta_bounds['lon_min']) &
    (pl.col('lon') <= atlanta_bounds['lon_max'])
)

print(f"\nValidation:")
print(f"  Test businesses in Atlanta bounds: {len(atlanta_test_biz):,}")
print(f"  Total test businesses: {len(test_biz_ids):,}")
print(f"  Atlanta percentage in test: {len(atlanta_test_biz)/len(test_biz_ids)*100:.1f}%")


GEOGRAPHIC DATA SPLITTING
  ✓ Backed up original features
  ✓ Using fixed features for splitting

Using processed directory: /Volumes/SunnySSD/Forkast_processed/ga
PHASE A4: GEOGRAPHIC DATA SPLITTING (NON-ATLANTA → ATLANTA)

Inputs:
  - /Volumes/SunnySSD/Forkast_processed/ga/features_ga.parquet
  - /Volumes/SunnySSD/Forkast_processed/ga/biz_ga.parquet

Creating output directories...
  ✓ /Volumes/SunnySSD/Forkast_processed/ga/xgboost_data

[Loading data...]
  XGBoost features: 4,363,086
  Businesses: 27,710

GEOGRAPHIC SPLIT: NON-ATLANTA TRAIN → ATLANTA TEST

[1/5] Identifying Atlanta businesses...
  Atlanta businesses: 6,887
  Total businesses: 27,710
  Atlanta percentage: 24.9%

[2/5] Splitting data geographically...
  Non-Atlanta samples (train/val): 3,349,419
  Atlanta samples (test): 1,013,667

[3/5] Temporal split of non-Atlanta data...
  Train samples: 2,758,345
  Val samples: 591,074
  Test samples: 1,013,667

[4/5] Temporal ranges:
  Train: 2005-12-09 00:00:00 to 2020-07-13 22:

## Phase B1: XGBoost Ranking Model Training

**What this does:**
- Loads split data and prepares features
- Trains XGBoost ranking model with `rank:pairwise` objective
- Evaluates using Recall@K, MRR, nDCG@K
- Generates feature importance analysis
- Creates transition probability matrix

**Expected time:** 20-40 minutes (depending on data size)


## Export Predictions for Visualization

**What this does:**
- Takes the generated XGBoost predictions
- Adds source and destination business metadata
- Formats columns to match LSTM prediction format exactly
- Exports to `outputs/atlanta_xgboost_predictions_with_meta.csv`

This ensures the visualization API can load XGBoost predictions with the same format as LSTM predictions.


In [ ]:
# Reload the module to get the latest changes
import importlib
import models.xgboost_ranker
importlib.reload(models.xgboost_ranker)
from models.xgboost_ranker import (
    load_data, prepare_features, train_model, predict_ranking, 
    evaluate_ranking, analyze_feature_importance, save_model_and_results,
    CATEGORICAL_FEATURES, NUMERICAL_FEATURES, BOOLEAN_FEATURES
)

print("=" * 80)
print("PHASE B1: XGBOOST RANKING MODEL TRAINING")
print("=" * 80)

# Load data
data_dir = PROCESSED_DIR / "xgboost_data"
train_df, val_df, test_df = load_data(data_dir)

print(f"\n✓ Data loaded:")
print(f"  Train: {len(train_df):,} pairs")
print(f"  Validation: {len(val_df):,} pairs")
print(f"  Test: {len(test_df):,} pairs")


PHASE B1: XGBOOST RANKING MODEL TRAINING

LOADING DATA

[1/3] Loading training data...
  ✓ 2,758,345 samples
  Positive: 655,825
  Negative: 2,102,520

[2/3] Loading validation data...
  ✓ 591,074 samples
  Positive: 143,977
  Negative: 447,097

[3/3] Loading test data...
  ✓ 1,013,667 samples
  Positive: 185,204
  Negative: 828,463

✓ Data loaded:
  Train: 2,758,345 pairs
  Validation: 591,074 pairs
  Test: 1,013,667 pairs


In [18]:
# Prepare features
print("\n[Step 1/5] Preparing features...")

X_train, y_train, group_train, feature_names = prepare_features(train_df)
X_val, y_val, group_val, _ = prepare_features(val_df)
X_test, y_test, group_test, _ = prepare_features(test_df)

print(f"\n✓ Features prepared:")
print(f"  Total features: {len(feature_names)}")
print(f"    - Categorical: {len(CATEGORICAL_FEATURES)}")
print(f"    - Numerical: {len(NUMERICAL_FEATURES)}")
print(f"    - Boolean: {len(BOOLEAN_FEATURES)}")
print(f"  Train shape: {X_train.shape}")
print(f"  Validation shape: {X_val.shape}")
print(f"  Test shape: {X_test.shape}")



[Step 1/5] Preparing features...

[Preparing ranking data with interaction features...]

  🔍 DIAGNOSTIC: Columns before create_interaction_features: 122
  🔗 Creating interaction features...
    Added 17 interaction features
  🔍 DIAGNOSTIC: Columns after create_interaction_features: 139
  🔍 DIAGNOSTIC: Found 17 interaction columns
  🔍 DIAGNOSTIC: Examples: ['distance_x_dst_rating', 'distance_x_rating_diff', 'price_x_rating', 'time_x_distance', 'user_pref_x_dst_rating']
      distance_x_dst_rating: mean=208.8553, std=370.2505, range=[0.0000, 2812.3706]
      distance_x_rating_diff: mean=2.6571, std=110.9482, range=[-2181.6138, 2203.1377]
      price_x_rating: mean=4.4586, std=3.0904, range=[0.0000, 20.0000]
      time_x_distance: mean=4474.6071, std=9703.8722, range=[0.0000, 91739.1388]
      user_pref_x_dst_rating: mean=15.9641, std=4.3548, range=[1.0000, 25.0000]
  - One-hot encoding categorical features...
  - Total features after encoding: 146
    Base features: 100
    Interaction 

In [19]:
# Show sample of feature names
print("\nSample feature names (showing enhanced features):")
print("\n  New Sentiment Features:")
sentiment_features = [f for f in feature_names if 'sentiment' in f.lower()]
for f in sentiment_features[:5]:
    print(f"    - {f}")

print("\n  New User Behavioral Features:")
user_features = [f for f in feature_names if f.startswith('user_')]
for f in user_features[:5]:
    print(f"    - {f}")

print("\n  New Operating Hours Features:")
hours_features = [f for f in feature_names if 'hours' in f.lower() or 'open' in f.lower()]
for f in hours_features[:5]:
    print(f"    - {f}")



Sample feature names (showing enhanced features):

  New Sentiment Features:
    - src_sentiment_score
    - dst_sentiment_score
    - sentiment_transition
    - sentiment_similarity
    - is_sentiment_upgrade

  New User Behavioral Features:
    - user_avg_rating
    - user_rating_std
    - user_total_reviews
    - user_unique_restaurants
    - user_visit_frequency

  New Operating Hours Features:
    - delta_hours
    - src_is_open_at_time
    - dst_is_open_at_time
    - hours_overlap
    - delta_hours_squared


In [ ]:
# Train model
print("\n[Step 2/5] Training XGBoost ranking model...")
print("  This may take 20-40 minutes depending on data size...")

model = train_model(
    X_train, y_train, group_train,
    X_val, y_val, group_val
)

print("\n✓ Model training complete!")


In [ ]:
# Generate predictions on test set
print("\n[Step 3/5] Generating predictions...")
predictions_df = predict_ranking(model, test_df, top_k=10)

print(f"\n✓ Predictions generated:")
print(f"  Total predictions: {len(predictions_df):,}")
print(f"  Unique source restaurants: {predictions_df['src_gmap_id'].n_unique():,}")
print(f"  Average predictions per source: {len(predictions_df)/predictions_df['src_gmap_id'].n_unique():.1f}")


In [ ]:
# Export predictions to CSV format compatible with visualization API
print("\n" + "=" * 80)
print("EXPORTING PREDICTIONS FOR VISUALIZATION")
print("=" * 80)

print("\n[1/8] Renaming gmap_id columns...")
# First, rename src_gmap_id and dst_gmap_id to match LSTM format
predictions_with_meta = predictions_df.rename({
    "src_gmap_id": "source_gmap_id",
    "dst_gmap_id": "dest_gmap_id",
    "distance_km": "link_distance_km",
    "same_category": "same_main_category"
})

print("[2/8] Adding prediction_id and row index...")
# Add prediction_id
predictions_with_meta = predictions_with_meta.with_row_index("prediction_id")

print("[3/8] Creating business indices...")
# Create business indices from gmap_id hashes for compatibility
# Use modulo to keep within Int32 range, or use Int64
predictions_with_meta = predictions_with_meta.with_columns([
    (pl.col("source_gmap_id").hash() % (2**31 - 1)).cast(pl.Int32()).alias("source_business_idx"),
    (pl.col("dest_gmap_id").hash() % (2**31 - 1)).cast(pl.Int32()).alias("dest_business_idx"),
])

# Add score columns if they don't exist (for compatibility with test data)
if "score_raw" not in predictions_with_meta.columns:
    print("[4/8] Computing normalized scores...")
    # Use absolute values for proper normalization (avoid negative percentages)
    predictions_with_meta = predictions_with_meta.with_columns([
        pl.col("score").alias("score_raw"),
        (pl.col("score").abs() / pl.col("score").abs().sum().over("source_gmap_id")).alias("score_share"),
        (pl.col("rank").cast(pl.Float64) / pl.count().over("source_gmap_id")).alias("score_cum_share"),
        pl.lit(None).cast(pl.Float64()).alias("score_z"),
    ])

print("[5/8] Adding source business metadata...")
# Join source business metadata
source_meta = biz_df.select([
    pl.col("gmap_id"),
    pl.col("name").alias("source_name"),
    pl.col("lat").alias("source_lat"),
    pl.col("lon").alias("source_lon"),
    pl.col("category_main").alias("source_category_main"),
    pl.col("category_all").list.join("|").alias("source_category_all"),
    pl.col("avg_rating").alias("source_avg_rating"),
    pl.col("num_reviews").alias("source_num_reviews"),
    pl.col("price_bucket").alias("source_price_bucket"),
    pl.col("is_closed").alias("source_is_closed"),
])

predictions_with_meta = predictions_with_meta.join(
    source_meta,
    left_on="source_gmap_id",
    right_on="gmap_id",
    how="left"
)

print("[6/8] Adding destination business metadata...")
# Join destination business metadata
dest_meta = biz_df.select([
    pl.col("gmap_id"),
    pl.col("name").alias("dest_name"),
    pl.col("lat").alias("dest_lat"),
    pl.col("lon").alias("dest_lon"),
    pl.col("category_main").alias("dest_category_main"),
    pl.col("category_all").list.join("|").alias("dest_category_all"),
    pl.col("avg_rating").alias("dest_avg_rating"),
    pl.col("num_reviews").alias("dest_num_reviews"),
    pl.col("price_bucket").alias("dest_price_bucket"),
    pl.col("is_closed").alias("dest_is_closed"),
])

predictions_with_meta = predictions_with_meta.join(
    dest_meta,
    left_on="dest_gmap_id",
    right_on="gmap_id",
    how="left"
)

print("[7/8] Selecting columns...")
# Select columns in the expected order (matching LSTM format)
final_columns = [
    "prediction_id",
    "source_business_idx",
    "source_gmap_id",
    "source_name",
    "source_lat",
    "source_lon",
    "source_category_main",
    "source_category_all",
    "source_avg_rating",
    "source_num_reviews",
    "source_price_bucket",
    "source_is_closed",
    "rank",
    "score_raw",
    "score_share",
    "score_cum_share",
    "score_z",
    "dest_business_idx",
    "dest_gmap_id",
    "dest_name",
    "dest_lat",
    "dest_lon",
    "dest_category_main",
    "dest_category_all",
    "dest_avg_rating",
    "dest_num_reviews",
    "dest_price_bucket",
    "dest_is_closed",
    "link_distance_km",
    "same_main_category",
]

# Keep only columns that exist
existing_cols = [col for col in final_columns if col in predictions_with_meta.columns]
predictions_export = predictions_with_meta.select(existing_cols)

print("[8/8] Saving to CSV...")
# Save to outputs directory
output_path = base_dir / "outputs" / "atlanta_xgboost_predictions_with_meta.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)
predictions_export.write_csv(output_path)

print(f"\n Exported predictions successfully!")
print(f"  File: {output_path}")
print(f"  Size: {output_path.stat().st_size / 1024 / 1024:.1f} MB")
print(f"  Total predictions: {len(predictions_export):,}")
print(f"  Unique sources: {predictions_export['source_business_idx'].n_unique():,}")
print(f"  Unique destinations: {predictions_export['dest_business_idx'].n_unique():,}")
print(f"  Columns: {len(predictions_export.columns)}")
print(f"\nColumns included: {existing_cols}")
print(f"\n💡 This file is now ready to be used by the visualization API!")
print(f"   The format matches LSTM predictions exactly.")


In [ ]:
# Evaluate model
print("\n[Step 4/5] Evaluating model performance...")
metrics = evaluate_ranking(predictions_df, k_values=[1, 5, 10])

print("\n✓ Evaluation complete:")
print("\n  Performance Metrics:")
for metric, value in metrics.items():
    print(f"    {metric}: {value:.4f}")

print("\n  Interpretation:")
print(f"    - Recall@10: {metrics.get('recall@10', 0):.1%} of actual next visits are in top-10 predictions")
print(f"    - MRR: {metrics.get('mrr', 0):.2f} average rank of first correct prediction")
print(f"    - nDCG@10: {metrics.get('ndcg@10', 0):.4f} ranking quality (higher is better)")


In [24]:
# Analyze feature importance
print("\n[Step 5/5] Analyzing feature importance...")
feature_importance = analyze_feature_importance(model, feature_names)

print("\n✓ Feature importance analysis complete!")
print("\n  Top feature categories:")
sorted_groups = sorted(feature_importance['group_totals'].items(), 
                      key=lambda x: x[1], reverse=True)
for group, total_score in sorted_groups[:5]:
    print(f"    {group:20s}: {total_score:8.4f}")



[Step 5/5] Analyzing feature importance...

FEATURE IMPORTANCE ANALYSIS

Top 20 Most Important Features:
   1. cuisine_pair_type_similar                6117.7104
   2. distance_bucket_20+km                    3751.9937
   3. dst_rating                               2805.7986
   4. same_category                            2609.3408
   5. cuisine_pair_type_diverse                2574.6482
   6. distance_km                              1311.1937
   7. is_rating_downgrade                      1175.1339
   8. rating_ratio                             1133.1189
   9. dst_is_highly_rated                      935.9001
  10. relative_results_rank                    934.7921
  11. is_in_relative_results                   905.5430
  12. rating_diff                              776.4322
  13. is_rating_upgrade                        731.9669
  14. user_pref_x_dst_rating                   641.0509
  15. distance_km_squared                      636.4898
  16. diversity_x_different_cat               

## Phase B2: Transition Matrix Generation

**What this does:**
- Generates A→B transition probability matrix for visualization
- Creates top-K predictions for each source restaurant
- Exports in multiple formats (Parquet, JSON, sparse matrix)

**Expected time:** 10-30 minutes (depending on number of restaurants)


In [ ]:
# Final summary
print("=" * 80)
print("PIPELINE COMPLETE!")
print("=" * 80)

print("\n✓ All phases completed successfully!")
print(f"\nModel Performance:")
print(f"  - Recall@10: {metrics.get('recall@10', 0):.4f}")
print(f"  - MRR: {metrics.get('mrr', 0):.4f}")
if 'ndcg@10' in metrics:
    print(f"  - nDCG@10: {metrics.get('ndcg@10', 0):.4f}")

print(f"\nData Locations:")
print(f"  - Processed data: {PROCESSED_DIR}")
print(f"  - Model data: {PROCESSED_DIR / 'xgboost_data'}")

print(f"\n Model is ready for evaluation and inference!")
print(f"   Key metrics indicate {'realistic' if metrics.get('recall@10', 0) < 0.9 else 'very high'} performance.")
